### Install necessary packages

In [1]:
import os
import json
import tempfile
import requests
import urllib
import pygsheets
import numpy as np
import pandas as pd
import asyncio
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from string import Template
from utils import write_base64str_obj_to_file, read_text_fromfile, read_env_variables

### Retrieve data using GraphQL

In [2]:
# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://gis-api.aiesec.org/graphql/?access_token=377a0dfb6e5f0126dbf76cab289c755c59e9578ef8acb9e2af72354e9a9a68a1")

async def getData():
    # Create a GraphQL client using the defined transport
    async with Client(transport=transport, fetch_schema_from_transport=True) as session:

        # Provide a GraphQL query
        query = gql(
            """
            query getApplicationList ($limit: Int, $start_date: DateTime, $end_date: DateTime){
            allOpportunityApplication(per_page: $limit, filters: {created_at: {from: $start_date, to: $end_date}}) {
                data {
                id
                status
                created_at
                date_matched
                date_pay_by_cash
                date_approved
                date_realized
                experience_start_date
                experience_end_date
                date_approval_broken
                nps_response_completed_at
                updated_at
                person {
                    id
                    full_name
                    home_mc {
                    name
                    }
                    home_lc {
                    name
                    }
                }
                host_lc {
                    name
                }
                host_mc: home_mc {
                    name
                }
                opportunity {
                    id
                    created_at
                    title
                    duration
                    sub_product {
                    name
                    }
                    programme {
                    short_name_display
                    }
                }
                standards {
                    option
                }
                }
            }
            }
        """
        )

        params = {	"mc_id": [518],
                    "start_date": "2021-01-01",
                    "end_date": "",
                    "limit": 1000
                }

        # Execute the query on the transport
        results = await session.execute(query, variable_values=params)
        # print(result)
        return results

results = await getData()

### Store response as a dataframe

In [3]:

# Reduce the dict by 3 Levels
results = results['allOpportunityApplication']['data']

#  Flatten dictionary and compress keys
results = pd.json_normalize(results, sep='_')

# results.replace([np.NaN, "", "-"], '', inplace=True)

## Examine the dataset

In [4]:
results.head()

,id,status,created_at,date_matched,date_pay_by_cash,date_approved,date_realized,experience_start_date,experience_end_date,date_approval_broken,...,person_home_lc_name,host_lc_name,host_mc_name,opportunity_id,opportunity_created_at,opportunity_title,opportunity_duration,opportunity_sub_product,opportunity_programme_short_name_display,opportunity_sub_product_name
0,6169117,rejected,2021-05-24T15:44:14Z,None,None,None,None,None,None,None,...,Awal,ESKISEHIR,Turkey,1261663,2020-06-28T14:50:18Z,Discover Turk'ish Culture | Develop your Marke...,None,NaN,GV,NaN
1,6169114,open,2021-05-24T15:41:10Z,None,None,None,None,None,None,None,...,Awal,SAMSUN,Turkey,1267025,2020-10-27T15:39:00Z,Practice Foreigner Language | Discuss World Is...,None,NaN,GV,NaN
2,6166570,open,2021-05-19T19:33:01Z,None,None,None,None,None,None,None,...,EASTERN MEDITERRANEAN,Tylos,Bahrain,1273447,2021-04-12T20:21:06Z,Children's Intercultural Environment Advocate,None,NaN,GV,NaN
3,6165784,matched,2021-05-18T15:16:07Z,2021-05-24T13:04:40Z,None,None,None,None,None,None,...,Dilmun,ESKISEHIR,Turkey,1269989,2021-02-04T12:03:23Z,Improve Animals’ Live quality | improve their ...,None,NaN,GV,NaN
4,6162477,open,2021-05-12T14:49:44Z,None,None,None,None,None,None,None,...,Awal,MC France,France,1274284,2021-05-04T13:30:34Z,HR Projects Manager,None,NaN,GTa,Other


In [5]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id                                        98 non-null     object 
 1   status                                    98 non-null     object 
 2   created_at                                98 non-null     object 
 3   date_matched                              2 non-null      object 
 4   date_pay_by_cash                          0 non-null      object 
 5   date_approved                             0 non-null      object 
 6   date_realized                             0 non-null      object 
 7   experience_start_date                     0 non-null      object 
 8   experience_end_date                       0 non-null      object 
 9   date_approval_broken                      0 non-null      object 
 10  nps_response_completed_at               

Create new columns for Easy Reading and Indices
* LC
* LC_ID*
* Department
* Partner_MC
* Partner_LC

*-Probably do need it for verification

In [6]:
# Create new multi-indices for grouping
new_cols = ['dept_prefix', 'lc', 'partner_mc', 'partner_lc']

def generate_new_fields(row):

    if row['person_home_mc_name'] == 'Bahrain':
        values = ['o', row['person_home_lc_name'],
                  row['host_mc_name'], row['host_lc_name']]
    else:
        values = ['i', row['host_lc_name'],
                  row['person_home_mc_name'], row['person_home_lc_name']]

    return dict(zip(new_cols, values))

results[new_cols] = results.apply(lambda row: generate_new_fields(row), axis=1, result_type='expand')
results[new_cols].head(10)

,dept_prefix,lc,partner_mc,partner_lc
0,o,Awal,Turkey,ESKISEHIR
1,o,Awal,Turkey,SAMSUN
2,i,Tylos,Turkey,EASTERN MEDITERRANEAN
3,o,Dilmun,Turkey,ESKISEHIR
4,o,Awal,France,MC France
5,o,Dilmun,Sri Lanka,COLOMBO CENTRAL
6,i,Tylos,Sri Lanka,NSBM
7,o,Awal,Turkey,SAMSUN
8,o,Awal,Brazil,SAO CARLOS
9,o,Tylos,Turkey,IZMIR


In [7]:
# Create a new field 'department' with incoming and outgoing labels as prefix
results['department'] = results.dept_prefix + results.opportunity_programme_short_name_display
results.drop('opportunity_programme_short_name_display', inplace=True, axis=1)
results['department']

0      oGV
1      oGV
2      iGV
3      oGV
4     oGTa
      ... 
93     oGV
94    oGTa
95     oGV
96     oGV
97    oGTa
Name: department, Length: 98, dtype: object

## Produce Performance Analytics DataFrame

* First convert dates from longform to YYYY-MM-DD
* Group by Date, LC, Dept, PartnerMC, PartnerLC, and the metrics like # of Applications, Accepted etc.. will be the aggregation

In [8]:
date_cols = ['created_at', 'date_matched', 'date_approved', 'date_realized', 'updated_at']
multi_indices = ['lc', 'department', 'partner_mc', 'partner_lc']
counting_by = ['id', 'person_id']

# Generate table with these columns only
perf_table = results[counting_by + date_cols + multi_indices].copy()

# Ensure that dates are uniform and shortened
perf_table.loc[:,date_cols] = results[date_cols].applymap(lambda x: x[:-10], na_action='ignore')


perf_table.head()


,id,person_id,created_at,date_matched,date_approved,date_realized,updated_at,lc,department,partner_mc,partner_lc
0,6169117,4652381,2021-05-24,None,None,None,2021-05-25,Awal,oGV,Turkey,ESKISEHIR
1,6169114,4652381,2021-05-24,None,None,None,2021-05-24,Awal,oGV,Turkey,SAMSUN
2,6166570,3021521,2021-05-19,None,None,None,2021-05-25,Tylos,iGV,Turkey,EASTERN MEDITERRANEAN
3,6165784,4563347,2021-05-18,2021-05-24,None,None,2021-05-24,Dilmun,oGV,Turkey,ESKISEHIR
4,6162477,584671,2021-05-12,None,None,None,2021-05-12,Awal,oGTa,France,MC France


In [9]:

def splitup_date_field(table: pd.DataFrame, remaining_fields: list, sel_date_col: str, metric_name: str):
    table = table[[sel_date_col, *remaining_fields, *counting_by]]
    _ = table.sort_values([sel_date_col, *remaining_fields])
    _.rename(columns={sel_date_col: "date", 
                     "id": metric_name+"~APP", 
                     "person_id": metric_name+"~PPL"}, inplace=True)
    
    
    return _.dropna(axis=0)


apps = splitup_date_field(perf_table, multi_indices, "created_at", "applications")
acc = splitup_date_field(perf_table, multi_indices, "date_matched", "accepted")

final = pd.concat([apps, acc])

### Push it to Google Sheets

In [10]:
# Credentials from service account file for Google Sheets
config_vars = read_env_variables()
print("Creating temporary file for service account credentials...")

temp = tempfile.NamedTemporaryFile()
try:
    access_creds = config_vars['GOOGLE_CREDS']
    
    write_base64str_obj_to_file(access_creds, temp.name)
finally:
    gc = pygsheets.authorize(service_file=temp.name)
    temp.close()

print("Uploading to Google Sheets...")
workbook = gc.open_by_key(config_vars["SPREADSHEET_ID"])
worksheet = workbook.worksheet_by_title(config_vars["SHEET_NAME"])

worksheet.set_dataframe(final, start='A1', copy_head=True)
print("Done!")

Creating temporary file for service account credentials...
Uploading to Google Sheets...
Done!
